<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-数据处理" data-toc-modified-id="1.-数据处理-1">1. 数据处理</a></span><ul class="toc-item"><li><span><a href="#1.1-One-hot-Encoding" data-toc-modified-id="1.1-One-hot-Encoding-1.1">1.1 One-hot Encoding</a></span></li><li><span><a href="#" data-toc-modified-id="-1.2"></a></span></li><li><span><a href="#1.2-时间序列" data-toc-modified-id="1.2-时间序列-1.3">1.2 时间序列</a></span></li><li><span><a href="#1.3-随机分配" data-toc-modified-id="1.3-随机分配-1.4">1.3 随机分配</a></span></li><li><span><a href="#1.4-缺失值补全" data-toc-modified-id="1.4-缺失值补全-1.5">1.4 缺失值补全</a></span></li><li><span><a href="#1.4-标准化" data-toc-modified-id="1.4-标准化-1.6">1.4 标准化</a></span></li></ul></li><li><span><a href="#2.-逻辑回归（不处理不平衡数据）" data-toc-modified-id="2.-逻辑回归（不处理不平衡数据）-2">2. 逻辑回归（不处理不平衡数据）</a></span></li><li><span><a href="#3.-逻辑回归（处理不平衡数据）" data-toc-modified-id="3.-逻辑回归（处理不平衡数据）-3">3. 逻辑回归（处理不平衡数据）</a></span></li></ul></div>

In [25]:
import pandas as pd 
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

## 1. 数据处理

### 1.1 One-hot Encoding

In [2]:
df = pd.read_pickle('../Data/IT_2018_数据处理.pkl')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56086 entries, 421101 to 1611872
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   loan_amnt             56086 non-null  float64       
 1   term                  56086 non-null  object        
 2   int_rate              55289 non-null  float64       
 3   grade                 56086 non-null  object        
 4   home_ownership        56086 non-null  object        
 5   verification_status   56086 non-null  object        
 6   loan_status           56086 non-null  object        
 7   purpose               56086 non-null  object        
 8   dti                   56086 non-null  float64       
 9   fico_range_high       56086 non-null  float64       
 10  open_acc              56086 non-null  float64       
 11  pub_rec               56086 non-null  float64       
 12  revol_bal             56086 non-null  float64       
 13  revol_uti

In [4]:
df['loan_status'] = df['loan_status'].apply(lambda s: float(s == 'Fully Paid'))

In [5]:
df['loan_status'].value_counts(normalize=False, dropna=False)

1.0    47254
0.0     8832
Name: loan_status, dtype: int64

In [6]:
dummy_list =['term', 'grade','home_ownership','verification_status','purpose']

In [7]:
df[dummy_list].isnull().any()

term                   False
grade                  False
home_ownership         False
verification_status    False
purpose                False
dtype: bool

In [8]:
df = pd.get_dummies(df, columns=dummy_list, drop_first=True)

In [9]:
df.shape

(56086, 49)

In [10]:
df.sample(5)

,loan_amnt,int_rate,loan_status,dti,fico_range_high,open_acc,pub_rec,revol_bal,revol_util,total_acc,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
450504,5000.0,7.34,1.0,28.59,804.0,7.0,0.0,26.0,0.1,21.0,...,0,0,1,0,0,0,0,0,0,0
1611456,12000.0,13.58,1.0,27.18,674.0,23.0,0.0,32094.0,49.8,39.0,...,0,0,0,0,0,0,0,0,0,0
455012,6200.0,9.43,1.0,15.14,669.0,18.0,0.0,5931.0,23.4,57.0,...,0,0,0,0,0,0,0,0,0,0
1527195,12000.0,11.05,1.0,9.98,699.0,9.0,1.0,13142.0,49.2,17.0,...,0,0,0,0,0,0,0,0,0,0
1475417,6000.0,7.84,1.0,13.26,699.0,7.0,0.0,5452.0,37.9,15.0,...,0,0,0,0,0,0,0,0,0,0


### 

**将原数据分为训练集和测试集**

此处教大家两种方式，一种是随机，一种是有时间序列的。

### 1.2 时间序列

In [11]:
df['IssueDate'].value_counts(normalize=False)

2018-01-01    8564
2018-03-01    7142
2018-04-01    6895
2018-02-01    6722
2018-05-01    6610
2018-06-01    4872
2018-07-01    4301
2018-08-01    3552
2018-09-01    2419
2018-10-01    2143
2018-11-01    1623
2018-12-01    1243
Name: IssueDate, dtype: int64

In [12]:
df_train = df.loc[(df['IssueDate'] < '2018-09-01')]
df_test =  df.loc[(df['IssueDate'] > '2017-08-01')]

In [13]:
df_train.drop(labels='IssueDate', axis=1, inplace=True)
df_test.drop(labels='IssueDate', axis=1, inplace=True)

D:\Temp\ipykernel_11612\1919218133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(labels='IssueDate', axis=1, inplace=True)


In [14]:
X_train = df_train.drop(['loan_status'], axis=1)
y_train = df_train.loc[:, 'loan_status']

X_test = df_test.drop(['loan_status'], axis=1)
y_test = df_test['loan_status']

### 1.3 随机分配

In [11]:
df.drop(labels='IssueDate', axis=1, inplace=True)

In [12]:
X = df.drop(['loan_status'], axis=1)
y = df.loc[:, 'loan_status']

**注意：对于不平衡的数据集，一定要stratify,否则非常容易出现训练集的y和测试集的y比例非常不对称**

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state=23,
                                                   shuffle=True,
                                                   stratify=y)

In [14]:
y_train.value_counts(normalize=True)

1.0    0.842526
0.0    0.157474
Name: loan_status, dtype: float64

In [15]:
y_test.value_counts(normalize=True)

1.0    0.842533
0.0    0.157467
Name: loan_status, dtype: float64

### 1.4 缺失值补全

In [16]:
# Create an imputer object with a median filling strategy
imputer = SimpleImputer(strategy='median')

# Train on the training features
imputer.fit(X_train)

# Transform both training and testing data
X_train = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

### 1.4 标准化

In [17]:
# Create an imputer object with a median filling strategy
scaler = StandardScaler()

# Train on the training features
scaler.fit(X_train)

# Transform both training and testing data
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

## 2. 逻辑回归（不处理不平衡数据）

In [20]:
logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X_train, y_train)

LogisticRegression()

In [21]:
y_pred = logreg.predict(X_test)
print('逻辑回归的准确率: {:.2f}'.format(logreg.score(X_test, y_test)))

逻辑回归的准确率: 0.84


In [22]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[   71  2137]
 [   55 11759]]


## 3. 逻辑回归（处理不平衡数据）

In [26]:
sm = SMOTE(random_state=12, n_jobs = -1, k_neighbors=5, sampling_strategy='auto')
X_train, y_train = sm.fit_resample(X_train, y_train)

In [27]:
logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X_train, y_train)

LogisticRegression()

In [28]:
y_pred = logreg.predict(X_test)
print('逻辑回归的准确率: {:.2f}'.format(logreg.score(X_test, y_test)))

逻辑回归的准确率: 0.64


In [29]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[1549  659]
 [4394 7420]]
